In [2]:
bRunStreetLightQueries = False

#!/usr/bin/python3

# Sample API client for StreetLight Data API
# This was tested with Python 3 on Ubuntu Linux 16.04.
# Refer to the StreetLight API documentation for more information.

import datetime as dt
import json
import requests
import sys
import time
import pandas as pd

now = dt.datetime.utcnow().isoformat()

# Edit the following two constants to a valid key and user login.
STL_KEY = 'KnsRn7hdgOQjbywguup5hBvCoMcRMNWy'
INSIGHT_LOGIN_EMAIL = 'analytics@wfrc.org'

ZONE_SET_NAME = "WasatchChoiceCenters_WFSmallAnalysisDists_v2"
MIDDLE_ZONE_SET_NAME = "Wasatch Front Bike Counters 30m Buffer"
ANALYSIS_NAME = "WF_Centers_SmallAreas_wBikeMdlFltr"

# create data ranges json object

dDateRanges = []

for y in range(2019, 2022): # end value not included in range
    # Create date ranges for each month of year in range
    for m in range(1, 13):
        dtMonthBeg = dt.datetime(y, m, 1)
        dtMonthEnd = dt.date(y + int(m / 12), (m % 12) + 1, 1) - dt.timedelta(days=1)
        dDateRanges.append(
            {
                'date_range_name': dtMonthBeg.strftime("%Y_%m"),
                'data_ranges'    : [{"start_date": dtMonthBeg.strftime("%m/%d/%Y") ,"end_date": dtMonthEnd.strftime("%m/%d/%Y")}]
            }
        )
        #print(dic)    
    # Add best months "BM" ranges for each year from April 1 through October 31
    dtBeg = dt.datetime(y, 4, 1)
    dtEnd = dt.datetime(y,10,31)
    dDateRanges.append(
        {
            'date_range_name': str(y) + "_BM",
            'data_ranges'    : [{"start_date": dtBeg.strftime("%m/%d/%Y") ,"end_date": dtEnd.strftime("%m/%d/%Y")}]
        }
    )
    # Add best year ranges for each year
    dtBeg = dt.datetime(y, 1, 1)
    dtEnd = dt.datetime(y,12,31)
    dDateRanges.append(
        {
            'date_range_name': str(y) + "_YR",
            'data_ranges'    : [{"start_date": dtBeg.strftime("%m/%d/%Y") ,"end_date": dtEnd.strftime("%m/%d/%Y")}]
        }
    )
    
dfDateRanges = pd.DataFrame(dDateRanges)
#display(dfDateRanges)

# create Modes df
dModes = [
            #['TRU','Truck'],
            ['BIC','Bicycle'],
            #['PED','Pedestrian'],
            #['BUS','Bus'],
            #['RAI','Rail'],
            #['ALL','All_Vehicles']
         ]

dfModes = pd.DataFrame(dModes, columns = ['mode_name','mode_description'])
#display(dfModes)

# create cross recordset for analysis
dfAnalysisSets = pd.DataFrame.merge(dfDateRanges, dfModes, how='cross')
dfAnalysisSets['analysis_set_name'] = ANALYSIS_NAME + '_' + dfAnalysisSets['date_range_name'].astype(str)

# rail and bus analysis periods only include April, May, September, and October of 2019 and 2020
for index, row in dfAnalysisSets.iterrows():
    if (row['mode_description']=='Rail' or row['mode_description']=='Bus'):
        if row['date_range_name'] not in ('2019_04','2019_05','2019_09','2019_10','2020_04','2020_05','2020_09','2020_10'):
            dfAnalysisSets.drop(index, inplace=True)

# add analysis toggle
dfAnalysisSets['RunQuery'] = True
#dfAnalysisSets['RunQuery'] = False
#dfAnalysisSets.loc[dfAnalysisSets['date_range_name'].str[-3:]=='_YR', 'RunQuery'] = True
#dfAnalysisSets.loc[dfAnalysisSets['analysis_set_name']=='WC_Centers_ODAnalysis_2019_01_ALL_revbins2', 'RunQuery'] = True
display(dfAnalysisSets)

def print_response(response):
    print("response code: {}".format(response.status_code))
    print("response body: {}".format(response.content))

date_range_name                                        data_ranges  \
0          2019_01  [{'start_date': '01/01/2019', 'end_date': '01/...   
1          2019_02  [{'start_date': '02/01/2019', 'end_date': '02/...   
2          2019_03  [{'start_date': '03/01/2019', 'end_date': '03/...   
3          2019_04  [{'start_date': '04/01/2019', 'end_date': '04/...   
4          2019_05  [{'start_date': '05/01/2019', 'end_date': '05/...   
5          2019_06  [{'start_date': '06/01/2019', 'end_date': '06/...   
6          2019_07  [{'start_date': '07/01/2019', 'end_date': '07/...   
7          2019_08  [{'start_date': '08/01/2019', 'end_date': '08/...   
8          2019_09  [{'start_date': '09/01/2019', 'end_date': '09/...   
9          2019_10  [{'start_date': '10/01/2019', 'end_date': '10/...   
10         2019_11  [{'start_date': '11/01/2019', 'end_date': '11/...   
11         2019_12  [{'start_date': '12/01/2019', 'end_date': '12/...   
12         2019_BM  [{'start_date': '04/01/2019', 'end_date': '10/...   
13         2019_YR  [{'start_date': '01/01/2019', 'end_date': '12/...   
14         2020_01  [{'start_date': '01/01/2020', 'end_date': '01/...   
15         2020_02  [{'start_date': '02/01/2020', 'end_date': '02/...   
16         2020_03  [{'start_date': '03/01/2020', 'end_date': '03/...   
17         2020_04  [{'start_date': '04/01/2020', 'end_date': '04/...   
18         2020_05  [{'start_date': '05/01/2020', 'end_date': '05/...   
19         2020_06  [{'start_date': '06/01/2020', 'end_date': '06/...   
20         2020_07  [{'start_date': '07/01/2020', 'end_date': '07/...   
21         2020_08  [{'start_date': '08/01/2020', 'end_date': '08/...   
22         2020_09  [{'start_date': '09/01/2020', 'end_date': '09/...   
23         2020_10  [{'start_date': '10/01/2020', 'end_date': '10/...   
24         2020_11  [{'start_date': '11/01/2020', 'end_date': '11/...   
25         2020_12  [{'start_date': '12/01/2020', 'end_date': '12/...   
26         2020_BM  [{'start_date': '04/01/2020', 'end_date': '10/...   
27         2020_YR  [{'start_date': '01/01/2020', 'end_date': '12/...   
28         2021_01  [{'start_date': '01/01/2021', 'end_date': '01/...   
29         2021_02  [{'start_date': '02/01/2021', 'end_date': '02/...   
30         2021_03  [{'start_date': '03/01/2021', 'end_date': '03/...   
31         2021_04  [{'start_date': '04/01/2021', 'end_date': '04/...   
32         2021_05  [{'start_date': '05/01/2021', 'end_date': '05/...   
33         2021_06  [{'start_date': '06/01/2021', 'end_date': '06/...   
34         2021_07  [{'start_date': '07/01/2021', 'end_date': '07/...   
35         2021_08  [{'start_date': '08/01/2021', 'end_date': '08/...   
36         2021_09  [{'start_date': '09/01/2021', 'end_date': '09/...   
37         2021_10  [{'start_date': '10/01/2021', 'end_date': '10/...   
38         2021_11  [{'start_date': '11/01/2021', 'end_date': '11/...   
39         2021_12  [{'start_date': '12/01/2021', 'end_date': '12/...   
40         2021_BM  [{'start_date': '04/01/2021', 'end_date': '10/...   
41         2021_YR  [{'start_date': '01/01/2021', 'end_date': '12/...   

   mode_name mode_description                           analysis_set_name  \
0        BIC          Bicycle  WF_Centers_SmallAreas_wBikeMdlFltr_2019_01   
1        BIC          Bicycle  WF_Centers_SmallAreas_wBikeMdlFltr_2019_02   
2        BIC          Bicycle  WF_Centers_SmallAreas_wBikeMdlFltr_2019_03   
3        BIC          Bicycle  WF_Centers_SmallAreas_wBikeMdlFltr_2019_04   
4        BIC          Bicycle  WF_Centers_SmallAreas_wBikeMdlFltr_2019_05   
5        BIC          Bicycle  WF_Centers_SmallAreas_wBikeMdlFltr_2019_06   
6        BIC          Bicycle  WF_Centers_SmallAreas_wBikeMdlFltr_2019_07   
7        BIC          Bicycle  WF_Centers_SmallAreas_wBikeMdlFltr_2019_08   
8        BIC          Bicycle  WF_Centers_SmallAreas_wBikeMdlFltr_2019_09   
9        BIC          Bicycle  WF_Centers_SmallAreas_wBikeMdlFltr_2019_10   
10       BIC    

In [3]:
#----------------------------------------------------------------------------------------
# Create a Zone Set.
#----------------------------------------------------------------------------------------

#ZONE_SET_REQUEST = {
#    "insight_login_email": INSIGHT_LOGIN_EMAIL,
#    "zone_set_name": ZONE_SET_NAME,
#    "zones": {
#        "type": "FeatureCollection",
#        "features": [
#            { "type": "Feature", "properties": { "id": 1, "name": "Mission", "is_pass": 0, "direction": None }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ -122.426698258661972, 37.769562689936315 ], [ -122.423394859041892, 37.772083876030344 ], [ -122.42225575572462, 37.770418101996292 ], [ -122.411206453547067, 37.769427623969634 ], [ -122.406991771273155, 37.769067446852375 ], [ -122.404656609472752, 37.767716767038294 ], [ -122.405169205965521, 37.762628984940662 ], [ -122.406593085112107, 37.760557752167074 ], [ -122.405795712790038, 37.758441432691242 ], [ -122.403232730326167, 37.75722564731312 ], [ -122.402549268335804, 37.751821915004783 ], [ -122.403346640657901, 37.749390106706535 ], [ -122.407561322931784, 37.748399347080543 ], [ -122.424875693354338, 37.74781389197571 ], [ -122.426698258661972, 37.769562689936315 ] ] ] ] } },
#            { "type": "Feature", "properties": { "id": 2, "name": "Financial District", "is_pass": 1, "direction": None }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ -122.405425504211919, 37.798033588378779 ], [ -122.398476973976571, 37.798933675051543 ], [ -122.396654408668923, 37.799698740100226 ], [ -122.397024617247041, 37.79929370664977 ], [ -122.396768319000643, 37.798911173018389 ], [ -122.395828558763895, 37.797898574431919 ], [ -122.393607307295227, 37.799113691070012 ], [ -122.392610591892605, 37.797876072083447 ], [ -122.394233814119715, 37.79690846455324 ], [ -122.393037755636598, 37.795670808599994 ], [ -122.3913575782436, 37.796278387822021 ], [ -122.390987369665496, 37.795693311702443 ], [ -122.392439726395011, 37.794500642185817 ], [ -122.389278714689581, 37.791462623416393 ], [ -122.401182344355107, 37.781965196242638 ], [ -122.405824190372982, 37.785701296639232 ], [ -122.406222876534017, 37.785723802695827 ], [ -122.407134159187834, 37.790337399578668 ], [ -122.404058580231194, 37.790764986655553 ], [ -122.405425504211919, 37.798033588378779 ] ] ] ] } }
#        ]
#    }
#}
#
#resp = requests.post(
#    'https://insight.streetlightdata.com/api/v2/zone_sets',
#    headers = {'content-type': 'application/json', 'x-stl-key': STL_KEY},
#    data = json.dumps(ZONE_SET_REQUEST))
#
#print_response(resp)
#
#if (resp.status_code == 201):
#    print("Created Zone Set successfully.")
#else:
#    print("Error creating Zone Set.")
#    sys.exit(1)
#
#


In [4]:
#----------------------------------------------------------------------------------------
# Create an O-D Analysis.
#----------------------------------------------------------------------------------------

if bRunStreetLightQueries:
    
    for index, row in dfAnalysisSets.iterrows():
        
        if row['RunQuery']==True:

            print(row['analysis_set_name'])

            CREATE_ANALYSIS_REQUEST = {
                "insight_login_email": INSIGHT_LOGIN_EMAIL,
                "analysis_name": row['analysis_set_name'],
                "analysis_type": "OD_MF_Analysis",
                "travel_mode_type": row['mode_description'],
                "description": "",
                "oz_sets": [{"name":ZONE_SET_NAME}],
                "dz_sets": [{"name":ZONE_SET_NAME}],
                "mfz_sets": [{"name":MIDDLE_ZONE_SET_NAME}],
                "date_ranges": row['data_ranges'],
                "day_types": "All Days|17, Weekday_TuWeTh|24, Weekend_SaSu|67",
                "day_parts": "All Day|0023, Early AM (12am-6am)|0005, Peak AM (6am-9am)|0608, Mid-Day (9am-3pm)|0914, Peak PM (3pm-6pm)|1517, Late PM (6pm-12am)|1823",
                "trip_attributes": True
            }

            # traveler attributes only exist for non-Truck modes
            if (row['mode_description']!='Truck'):
                CREATE_ANALYSIS_REQUEST['traveler_attributes'] = True
                #CREATE_ANALYSIS_REQUEST['truck_weight'] = 'Medium,Heavy'

            #print(CREATE_ANALYSIS_REQUEST)
            resp = requests.post(
                'https://insight.streetlightdata.com/api/v2/analyses',
                headers = {'content-type': 'application/json', 'x-stl-key': STL_KEY},
                data = json.dumps(CREATE_ANALYSIS_REQUEST))

            print_response(resp)

            if (resp.status_code == 201):
                print("Created Analysis successfully.")
                time.sleep(10) # wait ten seconds since URL has max limit of 3 calls per 15 seconds
            else:
                print("Error creating Analysis.")
                sys.exit(1)




In [5]:
#----------------------------------------------------------------------------------------
# Check the processing status of the Analysis.
#
# Note: depending on Analysis size, Analysis processing can take minutes to hours
# (for very large Analyses). A production integration should not block UI input
# from the end user while Analyses are processing, and it should resume
# gracefully when the client application is shut down and restarted while
# Analyses process.
#----------------------------------------------------------------------------------------


#pick a month that has all modes
dfOneOfEachMode = dfAnalysisSets[dfAnalysisSets['analysis_set_name'].str.contains("WF_Centers_SmallAreas_wBikeMdlFltr_2020_01")]


for index, row in dfOneOfEachMode.iterrows():

    print(row['analysis_set_name'])

    CHECK_STATUS_REQUEST = {
        "analyses":[{"name": row['analysis_set_name']}]
    }

    resp = requests.post(
        'https://insight.streetlightdata.com/api/v2/analyses/status',
        headers = {'content-type': 'application/json', 'x-stl-key': STL_KEY},
        data = json.dumps(CHECK_STATUS_REQUEST))

    print_response(resp)

    if (resp.status_code != 200):
        print("Error checking Analysis Status.")
        sys.exit(1)

    json_result = json.loads(resp.text)
    analysis_status = json_result["analyses"][0]["status"]

    if (analysis_status == "Available"):
        print("Analysis is Available!")
        time.sleep(10)
        break
    elif (analysis_status == "Processing"):
        print("Analysis is processing. Trying again after 1 minute...")
        time.sleep(10)
    else:
        print(analysis_status)
        time.sleep(10)
        #print("Error running Analysis.")
        #sys.exit(1)
#    
#    # only go through first time to get available downloades
#    break

WF_Centers_SmallAreas_wBikeMdlFltr_2020_01
response code: 200
response body: b'{"analyses":[{"metrics":["mf_bike","zone_mf_bike","mf_trip_bike","zone_trip_bike","mf_traveler_bike","zone_traveler_bike"],"name":"WF_Centers_SmallAreas_wBikeMdlFltr_2020_01","status":"Data Available","uuid":"14446498-44ea-4cdb-a45e-8ba0fe34dcec"}],"status":"success"}\n'
Data Available


In [6]:
dfOneOfEachMode

date_range_name                                        data_ranges  \
14         2020_01  [{'start_date': '01/01/2020', 'end_date': '01/...   

   mode_name mode_description                           analysis_set_name  \
14       BIC          Bicycle  WF_Centers_SmallAreas_wBikeMdlFltr_2020_01   

    RunQuery  
14      True

In [11]:
_keepfields         = ['Origin Zone ID','Middle Filter Zone ID','Destination Zone ID','Day Type','Day Part','Average Daily O-M-D Traffic (StL Index)','Average Daily Origin Zone Traffic (StL Index)','Average Daily Middle Filter Zone Traffic (StL Index)','Average Daily Destination Zone Traffic (StL Index)','Avg Travel Time (sec)']
_keepfields_newname = ['OID'           ,'MFID'                 ,'DID'                ,'day_type','day_part','avg_dy_omd_traffic'                     ,'avg_dy_orig_zone_traffic'                     ,'avg_dy_mdfl_zone_traffic'                            ,'avg_dy_dest_zone_traffic'                          ,'avg_travel_time_sec'  ]

dMetricInputDic = [
    ['TRU','mf_comm'],
    ['BUS','mf_bus' ],
    ['BIC','mf_bike'],
    ['PED','mf_ped' ],
    ['RAI','mf_rail'],
    ['ALL','mf_all' ]
]

dfMetricInputDic = pd.DataFrame(dMetricInputDic, columns = ['mode_name','metric_name'])
display(dfMetricInputDic)

mode_name metric_name
0       TRU     mf_comm
1       BUS      mf_bus
2       BIC     mf_bike
3       PED      mf_ped
4       RAI     mf_rail
5       ALL      mf_all

In [12]:
display(_keepfields)
display(_keepfields_newname)

['Origin Zone ID',
 'Middle Filter Zone ID',
 'Destination Zone ID',
 'Day Type',
 'Day Part',
 'Average Daily O-M-D Traffic (StL Index)',
 'Average Daily Origin Zone Traffic (StL Index)',
 'Average Daily Middle Filter Zone Traffic (StL Index)',
 'Average Daily Destination Zone Traffic (StL Index)',
 'Avg Travel Time (sec)']

['OID',
 'MFID',
 'DID',
 'day_type',
 'day_part',
 'avg_dy_omd_traffic',
 'avg_dy_orig_zone_traffic',
 'avg_dy_mdfl_zone_traffic',
 'avg_dy_dest_zone_traffic',
 'avg_travel_time_sec']

In [13]:
#----------------------------------------------------------------------------------------
# Get the O-D results.
#----------------------------------------------------------------------------------------

#"analyses":[{"metrics":["od_comm","zone_od_comm","od_trip_comm","zone_trip_comm"]

dfImport = pd.DataFrame()

for index, row in dfAnalysisSets.iterrows():

    print("Importing " + row['analysis_set_name'] + "...")

    _metric_name = dfMetricInputDic.loc[(dfMetricInputDic['mode_name']==row['mode_name']), 'metric_name'].values[0]
    
    resp = requests.get(
        'https://insight.streetlightdata.com/api/v2/analyses/download/name/{}'.format(row['analysis_set_name']) + '/' + _metric_name,
        headers = {'x-stl-key': STL_KEY})

    # resp.text contains results in CSV format.
    #print_response(resp)

    if (resp.status_code == 200):
        # Write results to a CSV file.        
        with open("temp.csv", "w") as csv_file:
            csv_file.write(resp.text)
        
        # read csv into temporary dataframe
        dfTemp = pd.read_csv("temp.csv",usecols=_keepfields)

        # rename columns to make shorter
        dfTemp.columns = _keepfields_newname

        # add columns with data from analysis set fields
        dfTemp['mode'      ] = row['mode_name'      ]
        dfTemp['date_range'] = row['date_range_name']

        # append temporary dataframe to Import dataframe
        dfImport = pd.concat([dfImport, dfTemp])

        # pause for 2 seconds due to API restrictions
        time.sleep(2)

        print("Done.")
    else:
        print("Error fetching O-D results.")
        sys.exit(1)

Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_01...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_02...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_03...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_04...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_05...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_06...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_07...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_08...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_09...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_10...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_11...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_12...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_BM...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_YR...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2020_01...
Done.
Importing WF_Centers_SmallAreas_wBikeMdlFltr_2020_02...
Done.
Importin

In [14]:
dfImport

OID  MFID  DID                    day_type  \
0      101    34  101          0: All Days (M-Su)   
1      101    34  101          0: All Days (M-Su)   
2      101    34  101          0: All Days (M-Su)   
3      101    34  101          0: All Days (M-Su)   
4      101    34  101  1:  Weekday_TuWeTh (Tu-Th)   
...    ...   ...  ...                         ...   
15439   91    40  221    2:  Weekend_SaSu (Sa-Su)   
15440   91    42  191          0: All Days (M-Su)   
15441   91    42  191          0: All Days (M-Su)   
15442   91    42  191    2:  Weekend_SaSu (Sa-Su)   
15443   91    42  191    2:  Weekend_SaSu (Sa-Su)   

                              day_part  avg_dy_omd_traffic  \
0               0: All Day (12am-12am)                  22   
1      2:  Peak AM (6am-9am) (6am-9am)                   3   
2      3:  Mid-Day (9am-3pm) (9am-3pm)                  15   
3      4:  Peak PM (3pm-6pm) (3pm-6pm)                   3   
4               0: All Day (12am-12am)                  25   
...                                ...                 ...   
15439  3:  Mid-Day (9am-3pm) (9am-3pm)                   1   
15440           0: All Day (12am-12am)                   0   
15441  3:  Mid-Day (9am-3pm) (9am-3pm)                   0   
15442           0: All Day (12am-12am)                   1   
15443  3:  Mid-Day (9am-3pm) (9am-3pm)                   1   

       avg_dy_orig_zone_traffic  avg_dy_mdfl_zone_traffic  \
0                         388.0                      34.0   
1                          65.0                       5.0   
2                         153.0                      26.0   
3                          98.0                       3.0   
4                         351.0                      39.0   
...                         ...                       ...   
15439                     391.0                      45.0   
15440                    1131.0                      72.0   
15441                     447.0                      30.0   
15442                     724.0                      75.0   
15443                     391.0                      42.0   

       avg_dy_dest_zone_traffic  avg_travel_time_sec mode date_range  
0                         381.0                 4243  BIC    2019_01  
1                          45.0                 5046  BIC    2019_01  
2                         170.0                 4183  BIC    2019_01  
3                         103.0                 3707  BIC    2019_01  
4                         333.0                 4107  BIC    2019_01  
...                         ...                  ...  ...        ...  
15439                    2076.0                 8643  BIC    2021_YR  
15440                     492.0                 8022  BIC    2021_YR  
15441                     180.0                 8022  BIC    2021_YR  
15442                     472.0                 8022  BIC    2021_YR  
15443                     220.0                 8022  BIC    2021_YR  

[255615 rows x 12 columns]

In [15]:
dfImport['day_type'] = dfImport['day_type'].str[0:1]
dfImport['day_part'] = dfImport['day_part'].str[0:1]
dfImport


OID  MFID  DID day_type day_part  avg_dy_omd_traffic  \
0      101    34  101        0        0                  22   
1      101    34  101        0        2                   3   
2      101    34  101        0        3                  15   
3      101    34  101        0        4                   3   
4      101    34  101        1        0                  25   
...    ...   ...  ...      ...      ...                 ...   
15439   91    40  221        2        3                   1   
15440   91    42  191        0        0                   0   
15441   91    42  191        0        3                   0   
15442   91    42  191        2        0                   1   
15443   91    42  191        2        3                   1   

       avg_dy_orig_zone_traffic  avg_dy_mdfl_zone_traffic  \
0                         388.0                      34.0   
1                          65.0                       5.0   
2                         153.0                      26.0   
3                          98.0                       3.0   
4                         351.0                      39.0   
...                         ...                       ...   
15439                     391.0                      45.0   
15440                    1131.0                      72.0   
15441                     447.0                      30.0   
15442                     724.0                      75.0   
15443                     391.0                      42.0   

       avg_dy_dest_zone_traffic  avg_travel_time_sec mode date_range  
0                         381.0                 4243  BIC    2019_01  
1                          45.0                 5046  BIC    2019_01  
2                         170.0                 4183  BIC    2019_01  
3                         103.0                 3707  BIC    2019_01  
4                         333.0                 4107  BIC    2019_01  
...                         ...                  ...  ...        ...  
15439                    2076.0                 8643  BIC    2021_YR  
15440                     492.0                 8022  BIC    2021_YR  
15441                     180.0                 8022  BIC    2021_YR  
15442                     472.0                 8022  BIC    2021_YR  
15443                     220.0                 8022  BIC    2021_YR  

[255615 rows x 12 columns]

In [16]:
import os
dfImport.to_csv(os.path.join('streetlight_smalldist_middlefilters_bicycle_data','centers_small_districts_w_bikemidfilters.csv'),index=False)